<a href="https://colab.research.google.com/github/dav-2/Text-to-Image-Generator-Flask-App-Created-Using-the-Stable-Diffusion-Model/blob/main/text_to_image_generator_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Text to Image Generator Flask App Created in Google Colab Using the Stable Diffusion Model.</h1>

Google Colab runs in a cloud-based environment where each notebook is executed on a virtual machine. This VM is isolated from the internet for security and privacy reasons.

Ngrok is a tool that creates secure tunnels to your localhost, allowing you to expose a local development server to the internet. When working in Google Colab, ngrok can be particularly useful for creating web applications or APIs that you want to test or share with others, as it creates tunnels that securely expose the services running inside these VMs to the internet.

To use ngrok it is needed to create an account and use the authtoken provided in the account.

To open the app, click on the ngrok-free.app link in the results at the end of the program.

In [ ]:
# Installing the packages
!pip install flask pyngrok
!pip install diffusers
!pip install transformers
!pip install accelerate scipy safetensors

# Authenticating ngrok
!ngrok authtoken # Write your ngrok authtoken here

In [ ]:
# Importing the libraries
from flask import Flask, request, send_file
import io
import torch
from diffusers import StableDiffusionPipeline
import time
from pyngrok import ngrok
import subprocess


# Creating the pipeline. stabilityai/stable-diffusion-2-1 offers a good balance beween precision, speed and resources needed, but there are other stable diffusion models available.
pipeline = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16
)

pipeline = pipeline.to('cuda') # Enabling Cuda

# Creating the Flask app
app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        prompt = request.form['prompt']
        image = pipeline(prompt=prompt).images[0]

        # Saving the image to a bytes buffer
        img_io = io.BytesIO()
        image.save(img_io, 'PNG')
        img_io.seek(0)
        return send_file(img_io, mimetype='image/png')
    return '''
        <form method="post">
            Enter a prompt: <input type="text" name="prompt"><br>
            <input type="submit" value="Generate Image">
        </form>
    '''


In [ ]:
# Starting ngrok
ngrok_process = subprocess.Popen(['ngrok', 'start', '--config', 'ngrok.yml', '--all'])

# Waiting for the ngrok process to initialise
time.sleep(5)

# Getting the public URL
if __name__ == "__main__":
  try:
    public_url = ngrok.connect(8081).public_url
    print(public_url)
    app.run(host="0.0.0.0", port=8081)
  finally:
    ngrok.disconnect(public_url=public_url)
